In [38]:
%load_ext autoreload
%autoreload 2
%matplotlib qt5
# Add modules to the path
import sys
import os
sys.path.insert(0, 'C:\\Users\\francescag\\Documents\\SourceTree_repos\\Python_git')
sys.path.insert(0, 'C:\\Users\\francescag\\Documents\\SourceTree_repos')
sys.path.insert(0, 'C:\\Users\\francescag\\Documents\\SourceTree_repos\\Python_git\\freely_moving_photometry_analysis')
import peakutils
from matplotlib import colors, cm
from scipy.signal import decimate
import copy
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import pandas as pd
import peakutils
import seaborn as sns
import pickle
from utils.plotting import HeatMapParams
from utils.plotting import heat_map_and_mean
from utils.plotting import get_photometry_around_event
from scipy import stats
from utils.plotting_visuals import makes_plots_pretty
from utils.value_change_utils import CustomAlignedDataRewardBlocks, plot_mean_trace_for_condition, get_all_experimental_records, get_block_change_info, add_traces_and_peaks, plot_mean_trace_for_condition_value_switch, one_session_get_block_changes

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
root_directory = 'C:\\Users\\francescag\\Documents\\SourceTree_repos\\Python_git\\freely_moving_photometry_analysis\\utils\\'
#figure_dir = 'W:\\photometry_2AC\\plots\\SNL_photo21_26\\value_change_experiment'
figure_dir = 'W:\\photometry_2AC\\plots\\SNL_photo28_35\\value_change_experiment'
processed_data_dir = os.path.join('W:\\photometry_2AC\\processed_data\\value_switch_data')
block_data_file = os.path.join(processed_data_dir, 'value_switch_all_tail_mice.csv')
all_reward_block_data = pd.read_pickle(block_data_file)

In [3]:
sorted_data = all_reward_block_data.sort_values(['mouse', 'session', 'trial number']).reset_index(drop=True)

In [4]:
sorted_data['block switches'] = sorted_data['block number'] - sorted_data['block number'].shift()

In [5]:
sorted_data['new sessions'] = sorted_data['session'].ne(sorted_data['session'].shift().bfill()).astype(int)
sorted_data.iloc[0, sorted_data.columns.get_loc('new sessions')] = 1
sorted_data.loc[sorted_data['new sessions'] == 1, 'block switches'] = 1


In [6]:
block_switch_inds = sorted_data.loc[sorted_data['block switches'] != 0].reset_index(drop=True)

In [7]:
# for each block I need to get: 
# - second half of trials (from ten trials after swtich)
# - traces corresponding to those trials
# - mean trace for those trials
# - block value
num_trials_to_look_at = 30
min_num_trials = 50
traces = []
peaks = []
trial_nums = []
block_inds = []
rel_reward_amounts = []
reward_amounts = []
mouse_ids = []
session_ids = []
num_blocks = block_switch_inds.index.values.shape[0]
for block_num, block in block_switch_inds.iterrows():
    first_trial = block['trial number']
    mouse = block['mouse']
    session = block['session']
    block_id = block['block number']
    all_session_trials = sorted_data[(sorted_data['mouse'] == mouse) & (sorted_data['session'] == session)]
    all_block_trials = all_session_trials[all_session_trials['block number'] == block_id]
    print(mouse, all_block_trials.shape[0])
    if block_id == 0 or all_block_trials.shape[0]>=min_num_trials:
        last_trials_of_block = all_block_trials[-num_trials_to_look_at:]
        mean_trace = last_trials_of_block['traces'].apply(np.mean)
        avg_trace = last_trials_of_block.groupby(['mouse', 'contra reward amount'])['traces'].apply(np.mean).values[0]
        decimated = decimate(avg_trace[int(len(avg_trace)/2):], 10) 
        peak_idx = peakutils.indexes(decimated)[0] 
        peak = decimated[peak_idx]
        traces.append(decimated)
        peaks.append(peak)
        trial_nums.append(last_trials_of_block['trial number'].values)
        rel_reward_amounts.append(last_trials_of_block['relative reward amount'].values[0])
        reward_amounts.append(last_trials_of_block['contra reward amount'].values[0])
        block_inds.append(last_trials_of_block.index.values[0])
        mouse_ids.append(mouse)
        session_ids.append(session)

SNL_photo37 40
SNL_photo37 90
SNL_photo37 48
SNL_photo37 118
SNL_photo37 45
SNL_photo37 74
SNL_photo37 47
SNL_photo37 79
SNL_photo43 44
SNL_photo43 133
SNL_photo43 42
SNL_photo43 108
SNL_photo43 44
SNL_photo43 90
SNL_photo43 39
SNL_photo43 90
SNL_photo44 49
SNL_photo44 69
SNL_photo44 39
SNL_photo44 107
SNL_photo44 40
SNL_photo44 90
SNL_photo44 45
SNL_photo44 85


In [8]:
avg_block_data = {}
avg_block_data['block id'] = block_inds
avg_block_data['peaks'] = peaks
avg_block_data['relative reward amount'] = rel_reward_amounts
avg_block_data['contra reward amount'] = reward_amounts
avg_block_data['mouse'] = mouse_ids
avg_block_data['session'] = session_ids
avg_block_dataf = pd.DataFrame(avg_block_data)

In [9]:
avg_block_dataf['avg traces'] = pd.Series(traces, index=avg_block_dataf.index)

In [10]:
df_for_plot1 = avg_block_dataf.groupby(['mouse', 'relative reward amount'])['peaks'].apply(np.mean)
df_for_plot2 = avg_block_dataf.groupby(['mouse', 'contra reward amount'])['peaks'].apply(np.mean)

In [11]:
df_for_plot1 = df_for_plot1.reset_index()
df_for_plot2 = df_for_plot2.reset_index()

In [12]:
df_for_plot1

,mouse,relative reward amount,peaks
0,SNL_photo37,-4.0,0.060790
1,SNL_photo37,0.0,0.490334
2,SNL_photo37,4.0,0.459256
3,SNL_photo43,-4.0,0.396902
4,SNL_photo43,0.0,0.230625
5,SNL_photo43,4.0,0.390294
6,SNL_photo44,-4.0,0.986330
7,SNL_photo44,0.0,1.461427
8,SNL_photo44,4.0,0.914569


In [13]:
df1 = df_for_plot1.pivot(index='relative reward amount', columns='mouse', values='peaks').sort_values('relative reward amount', ascending=False)

In [14]:
df2 = df_for_plot2.pivot(index='contra reward amount', columns='mouse', values='peaks').sort_values('contra reward amount', ascending=False)

In [15]:
df2

mouse,SNL_photo37,SNL_photo43,SNL_photo44
contra reward amount,,,
6.0,0.459256,0.390294,0.914569
2.0,0.404425,0.286051,1.303061


In [16]:
def francescas_plot(ax, data, show_err_bar=False, mean_linewidth=4, mean_line_color='blue', legend=False):
    """Plot lines for all mice from early to late.
    """
    data.plot(ax=ax, legend=False)
    data.mean(1).plot(ax=ax,linewidth=mean_linewidth,color=mean_line_color)

    if show_err_bar:
        yerr = data.std(axis=1)

        plt.errorbar(np.array([0,1]), data.mean(1), yerr, color=mean_line_color, linewidth=4)

    # drop the left and bottom spine for cool looking effect
    ax.spines['left'].set_position(('outward', 10))
    ax.spines['bottom'].set_position(('outward', 10))
    # Hide the right and top spines
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)

In [17]:
fig, ax = plt.subplots()
francescas_plot(ax, df2, mean_line_color='#7FB5B5', mean_linewidth=0, show_err_bar=False)

plt.ylabel('Z-scored fluorescence')
plt.tight_layout()

In [28]:
fig, ax = plt.subplots()
francescas_plot(ax, df1, mean_line_color='#7FB5B5', mean_linewidth=0, show_err_bar=False)

plt.ylabel('Z-scored fluorescence')
plt.tight_layout()

Old analysis


In [37]:
timepoints = all_reward_block_data['time points'].iloc[0]
font = {'size'   : 8}

matplotlib.rc('font', **font)

mouse_name = 'SNL_photo44'
session = '20210609'

fig, ax = plt.subplots(1,2, figsize=(4, 1.8)) #, figsize=(10,16))
possible_values = np.array([2, 6])
data = all_reward_block_data[(all_reward_block_data['mouse'] == mouse_name) & (all_reward_block_data['session'] == session)]
plot_mean_trace_for_condition_value_switch(ax[0], data, timepoints,
                              'contra reward amount', session, possible_values, error_bar_method='sem', save_location=processed_data_dir)
lg1 = ax[0].legend(title='Absolute \n value (ul)',loc='lower left', bbox_to_anchor=(0.7, 0.7),
            borderaxespad=0, frameon=False,prop={'size': 6 })
possible_values = np.array([-4, 0, 4])
plot_mean_trace_for_condition_value_switch(ax[1], data, timepoints,
                              'relative reward amount', session, possible_values, error_bar_method = 'sem', save_location=processed_data_dir)
lg2 = ax[1].legend(title='Relative \n value (ul)',loc='lower left', bbox_to_anchor=(0.7, 0.7),
            borderaxespad=0, frameon=False,prop={'size': 6 })
makes_plots_pretty([ax[0], ax[1]])
plt.tight_layout()


#plt.savefig(os.path.join(figure_dir, 'example_mouse{}_for_upgrade.pdf'.format(mouse_name)))


[1.4620e-03 4.6600e-04 1.3866e-02 1.0000e+00]
[0.987622 0.64532  0.039886 1.      ]
[0.578304 0.148039 0.404411 1.      ]
[0.987622 0.64532  0.039886 1.      ]


AttributeError: 'list' object has no attribute 'spines'

In [18]:
df1 = all_reward_block_data[['mouse','trial number','session','contra reward amount', 'peak size']]
df1 = df1.pivot_table(index=['mouse', 'contra reward amount'], values=['peak size'], aggfunc=np.nanmean)
df1 = df1.reset_index()
slope, intercept, r_val, p_val1, std_error = stats.linregress(df1['contra reward amount'], 
                                                             df1['peak size'])

In [19]:
df1

,mouse,contra reward amount,peak size
0,SNL_photo37,2.0,0.943270
1,SNL_photo37,6.0,0.986109
2,SNL_photo43,2.0,0.774721
3,SNL_photo43,6.0,0.676373
4,SNL_photo44,2.0,1.539850
5,SNL_photo44,6.0,1.482952


In [20]:
df2 = all_reward_block_data[['mouse','trial number','session','relative reward amount', 'peak size']]
df2 = df2.pivot_table(index=['mouse', 'relative reward amount'], values=['peak size'], aggfunc=np.nanmean)
df2 = df2.reset_index()
slope, intercept, r_val, p_val2, std_error = stats.linregress(df2['relative reward amount'], 
                                                             df2['peak size'])


In [21]:
df1_for_plot = df1.pivot(index='contra reward amount', columns='mouse', values='peak size').sort_values('contra reward amount', ascending=False)
df2_for_plot = df2.pivot(index='relative reward amount', columns='mouse', values='peak size').sort_values('relative reward amount', ascending=False)

In [22]:
fig, ax = plt.subplots()
francescas_plot(ax, df2_for_plot, mean_line_color='#7FB5B5', mean_linewidth=0, show_err_bar=False)
plt.legend()
plt.ylabel('Z-scored fluorescence')
plt.tight_layout()

In [23]:
fig, ax = plt.subplots()
francescas_plot(ax, df1_for_plot, mean_line_color='#7FB5B5', mean_linewidth=0, show_err_bar=False)
plt.legend()
plt.ylabel('Z-scored fluorescence')
plt.tight_layout()

In [24]:
df1_for_plot

mouse,SNL_photo37,SNL_photo43,SNL_photo44
contra reward amount,,,
6.0,0.986109,0.676373,1.482952
2.0,0.943270,0.774721,1.539850


In [28]:
font = {'size'   : 14}
matplotlib.rc('font', **font)

fig, axs = plt.subplots(2,1)
condition1 = 'contra reward amount'
sns.lineplot(ax=axs[0], x=condition1, y='peak size', data=df1, err_style='bars')
axs[0].text(4.5, 1.195, 'p-value = {0:.3f}'.format(p_val1))

condition2 = 'relative reward amount'
sns.lineplot(ax=axs[1], x=condition2, y='peak size', data=df2, err_style='bars')
axs[1].text(1.5, 1.3,'p-value = {0:.3f}'.format(p_val2))

for ax in axs:
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.set_ylabel('response (z-score)')
axs[0].set_xlabel('absolute value (ul)')
axs[1].set_xlabel('relative value (ul)')
plt.tight_layout()
plt.savefig(os.path.join(figure_dir, 'group_data_peaks_vs_value.pdf'))

In [13]:
plt.tight_layout()

In [14]:
colours = cm.inferno(np.linspace(0, 1, 5))
fig, ax = plt.subplots(1,1)
for trial_type in range(1,6):
    try:
        one_side_data = all_reward_block_data[trial_type-1].contra_data
        mean_trace = decimate(one_side_data.mean_trace, 10)
        time_points = decimate(one_side_data.time_points, 10)
        traces = decimate(one_side_data.sorted_traces, 10)
        ax.plot(time_points, mean_trace, lw=1.5, color=colours[trial_type-1], label= trial_type)
        ax.axvline(0, color='k')
    except:
        pass
plt.legend()


No handles with labels found to put in legend.


In [27]:
block_types = pd.DataFrame({'block type': [1, 2, 3, 4, 5], 'left reward': [6, 4, 2, 2, 2], 'right reward': [2, 2, 2, 4, 6]})

In [33]:
mouse_id = 'SNL_photo21'
sessions = ['20200917', '20200918', '20200921']
for session_num, date in enumerate(sessions):
    block_change_info, timepoints = one_session_get_block_changes(mouse_id, date, block_types)
    if session_num == 0:
        all_block_changes = block_change_info
    else:
         all_block_changes = pd.concat([all_block_changes,block_change_info], ignore_index=True)

In [43]:
block_change_info

,previous block type,new block type,first contra trial num,block change trial num,new reward amounts,change in reward amounts,new relative value,change in relative value,traces,peak size
0,1.0,4.0,19.0,19.0,4,2,2,6,"[-0.5358512698200665, -0.5364318153917825, -0....",2.018106
1,4.0,3.0,46.0,39.0,2,-2,0,-2,"[-0.9711220751151632, -0.9697295856545015, -0....",2.584282
2,3.0,1.0,60.0,59.0,2,0,-4,-4,"[-0.5416691840699284, -0.5419919626033652, -0....",4.572107
3,1.0,5.0,79.0,79.0,6,4,4,8,"[-0.5762941996276523, -0.5761260341446119, -0....",1.581628
4,5.0,2.0,99.0,99.0,2,-4,-2,-6,"[-1.2512422365874305, -1.2509320018301078, -1....",NaN
5,2.0,5.0,122.0,119.0,6,4,4,6,"[1.4457455953003535, 1.4463447984178535, 1.446...",NaN
6,5.0,4.0,139.0,139.0,4,-2,2,-2,"[-0.5041475111146595, -0.5030853983883056, -0....",4.281052
7,4.0,5.0,160.0,159.0,6,2,4,2,"[-0.9325599239490393, -0.9324308633048524, -0....",1.434236
8,5.0,4.0,180.0,179.0,4,-2,2,-2,"[0.5130707078453702, 0.5126761816570159, 0.512...",NaN
9,4.0,1.0,199.0,199.0,2,-2,-4,-6,"[1.6414452537223487, 1.6417079832128545, 1.641...",-0.403120


In [41]:
fig, axs = plt.subplots(1,1)

In [42]:
plot_mean_trace_for_condition(axs, all_block_changes, timepoints, 'new reward amounts')
plot_mean_trace_for_condition(axs, all_block_changes, timepoints, 'change in reward amounts')
plot_mean_trace_for_condition(axs, all_block_changes, timepoints, 'new relative value')
plot_mean_trace_for_condition(axs, all_block_changes, timepoints, 'change in relative value')

KeyError: 'mouse'

In [35]:
all_block_changes

,previous block type,new block type,first contra trial num,block change trial num,new reward amounts,change in reward amounts,new relative value,change in relative value,traces,peak size
0,5.0,1.0,21.0,19.0,2,-4,-4,-8,"[-0.7299424890039922, -0.7297226727881596, -0....",-0.271436
1,1.0,5.0,41.0,39.0,6,4,4,8,"[0.44878713651517876, 0.44821938348846324, 0.4...",NaN
2,5.0,1.0,62.0,59.0,2,-4,-4,-8,"[-0.8877375543387853, -0.8878435904235363, -0....",NaN
3,1.0,4.0,83.0,79.0,4,2,2,6,"[0.6966704852891275, 0.6974044942385212, 0.698...",NaN
4,4.0,3.0,100.0,99.0,2,-2,0,-2,"[-1.9407019201346587, -1.9399484326377192, -1....",0.174011
5,3.0,5.0,119.0,119.0,6,4,4,4,"[-0.4517721097313717, -0.45147117634062783, -0...",NaN
6,5.0,1.0,141.0,139.0,2,-4,-4,-8,"[-1.2610607438061245, -1.2612758543722067, -1....",1.726531
7,1.0,4.0,160.0,159.0,4,2,2,6,"[-0.7715575997160236, -0.7714038825922299, -0....",0.554537
8,4.0,2.0,179.0,179.0,2,-2,-2,-4,"[-1.4442105623118027, -1.4430205401828489, -1....",-0.573510
9,2.0,1.0,201.0,199.0,2,0,-4,-2,"[-1.296963043831093, -1.2971317655206531, -1.2...",0.948058


In [29]:
fig, axs = plt.subplots(nrows=2,ncols=2)
sns.pointplot(x='new reward amounts', y='peak size', data=all_block_changes, ax=axs[0,0], ci='sd')
sns.pointplot(x='change in reward amounts', y='peak size', data=block_change_info, ax=axs[0,1], ci='sd')
sns.pointplot(x='new relative value', y='peak size', data=block_change_info, ax=axs[1,0], ci='sd')
sns.pointplot(x='change in relative value', y='peak size', data=block_change_info, ax=axs[1,1], ci='sd')

In [65]:
colours = cm.inferno(np.linspace(0,1, 5))
fig2, ax = plt.subplots(1,3)
for trial_type in range(1,6):
    try:
        one_side_data = all_reward_block_data[trial_type-1].contra_data
        x_vals = np.ones([len(one_side_data.trial_peaks)])*trial_type
        ax[0].hist(one_side_data.trial_peaks, color=colours[trial_type-1], alpha=0.4)
        ax[1].scatter(trial_type, np.mean(one_side_data.trial_peaks), color=colours[trial_type-1])
        ax[1].set_ylabel('Mean peak')
        ax[2].scatter(trial_type, np.median(one_side_data.trial_peaks), color=colours[trial_type-1])
        ax[2].set_ylabel('Median peak')
        print(np.mean(one_side_data.trial_peaks), np.median(one_side_data.trial_peaks))
    except:
        pass

0.8719974503007026 0.8838530247845447
0.734418515406605 0.5818760037034908
1.3165566302983154 1.1614116229862628
0.7800667694836998 0.663554277871089
0.8304953025800229 0.5631001717818181


In [217]:
np.median(np.asarray(all_trial_type_data[-2].contra_data.trial_peaks)/4)

0.15315428214827714

In [ ]:
np.median(np.asarray(all_trial_type_data[-2].contra_data.trial_peaks)/4)

In [209]:
np.asarray(one_side_data.trial_peaks)/4

array([0.14386391, 0.36703822, 0.03294286, 0.67309225, 0.1799292 ,
       0.26867563, 0.06190406])

In [199]:
np.median([0.13636681, 0.46637622, 0.51340394])

0.46637622

In [207]:
np.median(np.concatenate([np.asarray(one_side_data.trial_peaks),np.asarray([0.13636681, 0.46637622, 0.51340394])]))

0.5444297804051131